In [2]:
import numpy as np
import networkx as nx
import karateclub as kc
from sklearn.metrics import roc_auc_score
import math

In [79]:
from itertools import combinations
from typing import Any, Callable, Dict, Iterator, Tuple

ScoreFunc = Callable[[int, int, nx.Graph], float]


def preferential_attachment(n: int, m: int, graph: nx.Graph) -> float:
    return graph.degree(n) * graph.degree(m)


def common_neighbors(n: int, m: int, graph: nx.Graph) -> float:
    return len(set(graph.neighbors(n)) & set(graph.neighbors(m)))


def jacquard(n: int, m: int, graph: nx.Graph) -> float:
    n_n, n_m = set(graph.neighbors(n)), set(graph.neighbors(m))
    return len(n_n & n_m) / len(n_n | n_m)


def adamic_adar(n: int, m: int, graph: nx.Graph) -> float:
    neighbors = set(graph.neighbors(n)) & set(graph.neighbors(m))
    return sum(
        1 / math.log(graph.degree(k)) if graph.degree(k) > 1 else 0 for k in neighbors
    )


def random(n: int, m: int, graph: nx.Graph) -> float:
    return np.random.uniform(0, 1)


def missing_links(graph: nx.Graph) -> Iterator[Tuple[int, int]]:
    return filter(lambda ns: not graph.has_edge(*ns), combinations(graph.nodes, 2))


def evaluate(
    graph: nx.Graph,
    score: ScoreFunc,
    true_links: np.ndarray,
    k: int = 8,
    T: int = 1,
    print_links: bool = False,
    kwargs_func: Callable[[nx.Graph], Dict[str, Any]] = lambda g: {}
):
    top_score = acc = auroc = 0
    for _ in range(T):
        missing = np.array(list(missing_links(graph)))

        kwargs = kwargs_func(graph)
        scores = np.array([(n, m, score(n, m, graph, **kwargs)) for n, m in missing])

        # top_idxs = np.argpartition(scores[:, 2], -k)[-k:]
        top_idxs = np.argsort(scores[:, 2])[::-1][:k]

        if print_links:
            print("Links:")
            print(missing[top_idxs] + 1)

        top_score += scores[top_idxs].sum()
        acc += true_links[top_idxs].sum()
        auroc += roc_auc_score(true_links, scores[:, 2])

    print(score.__name__)
    if T > 1:
        print("Averaged over", T, "trials")
    print("Top", k, "Score:", top_score / T)
    print("Accuracy:", acc / T, "/", k)
    print("AUROC:", auroc / T)
    print()

In [71]:
# Zero index karate club graph
graph = nx.karate_club_graph()
rm_links = [
    (i - 1, j - 1)
    for i, j in [
        (1, 5),
        (2, 4),
        (3, 29),
        (6, 17),
        (9, 34),
        (16, 33),
        (24, 26),
        (25, 32),
    ]
]
graph.remove_edges_from(rm_links)
true_links = np.array([(n, m) in rm_links for n, m in missing_links(graph)])

In [ ]:
score_funcs = [preferential_attachment, common_neighbors, jacquard, adamic_adar]

evaluate(graph, random, true_links, T=100)
for score in score_funcs:
    evaluate(graph, score, true_links)

In [ ]:
def deep_walk_score(n: int, m: int, graph: nx.Graph, x: np.ndarray = None) -> float:
    return x[n] @ x[m]


def deep_walk_embeddings(graph: nx.Graph) -> Dict[str, Any]:
    deep_walk = kc.DeepWalk(dimensions=2)
    deep_walk.fit(graph)
    return {"x": deep_walk.get_embedding()}

In [104]:
import dcsbm


def dcsbm_params(graph: nx.Graph) -> Dict[str, Any]:
    A = nx.to_numpy_array(graph, weight=None) - np.identity(graph.number_of_nodes())
    g = dcsbm.regularized_spectral_clustering(A, 2)
    print(g)
    _, P = dcsbm.parameter_estimation(A, g)
    print(P)
    return {"g": g, "P": P}


def dcsbm_score(
    n: int, m: int, graph: nx.Graph, g: np.ndarray = None, P: np.ndarray = None
) -> float:
    return P[g[n], g[m]]

In [ ]:
import latent_space_model as lsm



In [80]:
evaluate(
    graph, deep_walk_score, true_links, k=10, T=10, kwargs_func=deep_walk_embeddings
)

deep_walk_score
Averaged over 10 trials
Top 10 Score: 562.9316980361939
Accuracy: 1.6 / 10
AUROC: 0.7029503105590063



In [106]:
evaluate(graph, dcsbm_score, true_links, k=10, kwargs_func=dcsbm_params)

[0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1]
[[1.68136611 0.25761603]
 [0.25761603 1.80886612]]
dcsbm_score
Top 10 Score: 462.0886611717532
Accuracy: 0.0 / 10
AUROC: 0.661620082815735



C:\Users\Aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
